In [2]:
import os
from tqdm import tqdm # Para barras de progresso

from typing import List, Optional, Union, Dict

from haystack import Document
from haystack import Pipeline

from haystack_integrations.document_stores.chroma import ChromaDocumentStore
from haystack_integrations.components.embedders.ollama import OllamaDocumentEmbedder
from haystack_integrations.components.embedders.ollama import OllamaTextEmbedder
from haystack_integrations.components.retrievers.chroma import ChromaQueryTextRetriever
from haystack_integrations.components.retrievers.chroma import ChromaEmbeddingRetriever

from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.converters import PyPDFToDocument
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy

from mysql.connector import connect
from mysql.connector.connection import MySQLConnection
from mysql.connector.cursor import MySQLCursor
from mysql.connector.cursor import MySQLCursorAbstract
from mysql.connector import Error
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector.abstracts import MySQLConnectionAbstract

MYSQL_HOST = os.getenv("MYSQL_HOST", "localhost")
MYSQL_PORT = int(os.getenv("MYSQL_PORT", 3306))
MYSQL_USER = os.getenv("MYSQL_USER", "root")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD", "root")
MYSQL_DB = os.getenv("MYSQL_DB", "board_games_db")

In [ ]:
def get_mysql_connection() -> MySQLConnectionPool | MySQLConnectionAbstract:
  print(f"Conectando ao MySQL: host={MYSQL_HOST}, port={MYSQL_PORT}, user={MYSQL_USER}, db={MYSQL_DB}")
  return connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    database=MYSQL_DB,
    port=MYSQL_PORT
  )

def fetch_boardgames_from_mysql(conn: MySQLConnectionAbstract) -> list[dict]:
  print("Buscando jogos do MySQL...")
  cursor: MySQLCursorAbstract = conn.cursor(dictionary=True)

  query: str = """
    SELECT 
        jd.id, 
        jd.nm_jogo, 
        jd.idade_minima, 
        jd.qt_favorito, 
        jd.qt_jogadores_max,
        jd.qt_jogadores_min,
        jd.qt_jogou,
        jd.qt_quer,
        jd.qt_tem,
        jd.qt_teve,
        jd.thumb,
        jd.tp_jogo,
        jd.vl_tempo_jogo,
        GROUP_CONCAT(DISTINCT c.nm_categoria SEPARATOR ', ') as categorias,
        GROUP_CONCAT(DISTINCT t.nm_tema SEPARATOR ', ') as temas,
        GROUP_CONCAT(DISTINCT a.nm_profissional SEPARATOR ', ') as artistas,
        GROUP_CONCAT(DISTINCT m.nm_mecanica SEPARATOR ', ') as mecanicas
    FROM jogo_detalhado jd
    LEFT JOIN jogo_categoria jc ON jc.jogo_id = jd.id
    LEFT JOIN categoria c ON c.id = jc.categoria_id
    LEFT JOIN jogo_tema jt ON jt.jogo_id = jd.id
    LEFT JOIN tema t ON jt.tema_id = t.id
    LEFT JOIN jogo_artista ja ON ja.jogo_id = jd.id
    LEFT JOIN artista a ON a.id = ja.artista_id
    LEFT JOIN jogo_mecanica jm ON jm.jogo_id = jd.id
    LEFT JOIN mecanica m ON jm.mecanica_id = m.id
    WHERE jd.id IS NOT NULL AND jd.nm_jogo IS NOT NULL AND jd.nm_jogo != ''
    GROUP BY jd.id, jd.nm_jogo, jd.idade_minima, jd.qt_favorito, jd.qt_jogadores_max,
              jd.qt_jogadores_min, jd.qt_jogou, jd.qt_quer, jd.qt_tem, jd.qt_teve,
              jd.thumb, jd.tp_jogo, jd.vl_tempo_jogo
    LIMIT 1000;
  """
  
  cursor.execute(query)
  results: List[RowType | Dict[str, RowItemType]] = cursor.fetchall()
  cursor.close()
  print(f"Total de {len(results)} jogos buscados do MySQL.")
  return results

mysql_conn: MySQLConnectionAbstract = get_mysql_connection()
boardgames: list[dict] = fetch_boardgames_from_mysql(mysql_conn)
mysql_conn.close()

print(f"Total de jogos encontrados: {len(boardgames)}")
print(f"Exemplo de jogo: {boardgames[0]}")

Conectando ao MySQL: host=localhost, port=3306, user=root, db=board_games_db
Buscando jogos do MySQL...
Total de 1000 jogos buscados do MySQL.
Total de jogos encontrados: 1000
Exemplo de jogo: {'id': 1, 'nm_jogo': 'Old Dragon 2ª Edição: Livro I - Regras Básicas', 'idade_minima': 12, 'qt_favorito': 69, 'qt_jogadores_max': 10, 'qt_jogadores_min': 1, 'qt_jogou': 74, 'qt_quer': 57, 'qt_tem': 204, 'qt_teve': 1, 'thumb': 'https://storage.googleapis.com/ludopedia-capas/43761_t.jpg', 'tp_jogo': 'b', 'vl_tempo_jogo': 120, 'categorias': None, 'temas': 'Aventura, Fantasia, Literatura, Medieval', 'artistas': None, 'mecanicas': 'Narração de Histórias, Papel e Caneta, Rolagem de Dados, RPG'}


In [5]:
def prepare_haystack_documents(boardgames_data: list[dict]) -> list[Document]:
    """
    Prepara uma lista de Documentos Haystack a partir dos dados dos jogos de tabuleiro.
    Cada Documento contém informações relevantes sobre o jogo, como nome, tipo, idade mínima,
    número de jogadores, tempo de jogo, categorias, temas, mecânicas e artistas.
    
    Args:
        boardgames_data (list[dict]): Lista de dicionários contendo os dados dos jogos de tabuleiro.
    
    Returns:
        list[Document]: Lista de Documentos Haystack preparados.
    """
    
    print("Preparando Documentos Haystack...")
    haystack_docs: list[Document] = []

    for game in tqdm(boardgames_data, desc="Convertendo dados para Documentos"):
        content_parts: list[str] = [
            f"Nome do jogo: {game.get('nm_jogo', 'N/A')}.",
            f"Tipo: {game.get('tp_jogo', 'N/A')}.",
            f"Adequado para maiores de {game.get('idade_minima', 'N/A')} anos.",
            f"Pode ser jogado por {game.get('qt_jogadores_min', 'N/A')} a {game.get('qt_jogadores_max', 'N/A')} jogadores.",
            f"Tempo médio de jogo: {game.get('vl_tempo_jogo', 'N/A')} minutos.",
            f"Descrição: {game.get('descricao', 'N/A')}." if game.get('descricao') else "Descrição não disponível.",
            f"O jogo se baseia nas seguintes categorias: {game.get('categorias', 'N/A')}." if game.get('categorias') else "Categorias não disponíveis.",
            f"O jogo aborda os seguintes temas: {game.get('temas', 'N/A')}." if game.get('temas') else "Temas não disponíveis.",
            f"O jogo utiliza as seguintes mecânicas: {game.get('mecanicas', 'N/A')}." if game.get('mecanicas') else "Mecânicas não disponíveis.",
        ]
        
        if game.get('categorias'):
            content_parts.append(f"Categorias: {game['categorias']}.")
        if game.get('temas'):
            content_parts.append(f"Temas: {game['temas']}.")
        if game.get('mecanicas'):
            content_parts.append(f"Mecânicas: {game['mecanicas']}.")
        if game.get('artistas'):
            content_parts.append(f"Artistas: {game['artistas']}.")

        content: str = " ".join(content_parts)

        meta: Dict[str, Optional[Union[str, int]]] = {
            "mysql_id": game.get('id'),
            "title": game.get('nm_jogo', 'N/A'), # 'title' é um campo comum para meta
            "min_age": game.get('idade_minima'),
            "max_players": game.get('qt_jogadores_max'),
            "min_players": game.get('qt_jogadores_min'),
            "play_time_minutes": game.get('vl_tempo_jogo'),
            "thumbnail": game.get('thumb'),
            "game_type": game.get('tp_jogo'),
            "categories_list": game.get('categorias', '').split(', ') if game.get('categorias') else [],
            "themes_list": game.get('temas', '').split(', ') if game.get('temas') else [],
            "mechanics_list": game.get('mecanicas', '').split(', ') if game.get('mecanicas') else [],
            "artists_list": game.get('artistas', '').split(', ') if game.get('artistas') else [],
            "favorite_count": game.get('qt_favorito'),
            "played_count": game.get('qt_jogou'),
            "want_count": game.get('qt_quer'),
            "have_count": game.get('qt_tem'),
            "had_count": game.get('qt_teve')
        }
        
        # Remove chaves com valor None dos metadados para evitar problemas com alguns DocumentStores
        meta_cleaned: Dict[str, str | int] = {k: v for k, v in meta.items() if v is not None}

        haystack_docs.append(Document(content=content, meta=meta_cleaned))

    print(f"Total de {len(haystack_docs)} Documentos Haystack preparados.")
    if haystack_docs:
        print("\nAmostra do primeiro Documento Haystack:")
        print(f"Content: {haystack_docs[0].content[:500]}...")
        print(f"Meta: {haystack_docs[0].meta}")
    return haystack_docs

In [ ]:
print("Inicializando o ChromaDocumentStore...")

document_store_for_raw_boardgames: ChromaDocumentStore = ChromaDocumentStore(persist_path="./test_databases/chroma_db_raw_boardgames", collection_name="raw_boardgames")

print(f"ChromaDocumentStore inicializado com persist_path: {document_store_for_raw_boardgames._persist_path} e collection_name: {document_store_for_raw_boardgames._collection_name}")

print(f"ChromaDocumentStore usando path: ./test_databases/chroma_db_raw_boardgames. Documentos existentes: {document_store_for_raw_boardgames.count_documents()}")

Inicializando o ChromaDocumentStore...
ChromaDocumentStore inicializado com persist_path: ./chroma_db_raw_boardgames e collection_name: raw_boardgames
ChromaDocumentStore usando path: ./chroma_db_raw_boardgames. Documentos existentes: 1000


In [12]:
# Embedders
document_embedder: OllamaDocumentEmbedder = OllamaDocumentEmbedder(
  model="nomic-embed-text",
  url="http://localhost:11434",
  # Você pode adicionar outros parâmetros como timeout, etc.
      # generation_kwargs={"num_gpu": 1} # Exemplo se quiser especificar uso de GPU no Ollama
)

text_embedder: OllamaTextEmbedder = OllamaTextEmbedder(
  model="nomic-embed-text",
  url="http://localhost:11434",
  # Você pode adicionar outros parâmetros como timeout, etc.
      # generation_kwargs={"num_gpu": 1} # Exemplo se quiser especificar uso de GPU no Ollama
)

In [24]:
# Retrievers
query_text_retriever: ChromaQueryTextRetriever = ChromaQueryTextRetriever(
    document_store=document_store_for_raw_boardgames,
)

embedding_retriever: ChromaEmbeddingRetriever = ChromaEmbeddingRetriever(
    document_store=document_store_for_raw_boardgames,
)

In [ ]:
try:
  connection_mysql: MySQLConnectionAbstract = get_mysql_connection()
  raw_boardgames: list[dict] = fetch_boardgames_from_mysql(connection_mysql)
  haystack_docs: list[Document] = prepare_haystack_documents(raw_boardgames)
finally:
  if connection_mysql and connection_mysql.is_connected():
    connection_mysql.close()
    print("Conexão MySQL fechada.")
  
if not haystack_docs:
    print("Nenhum documento Haystack foi preparado. Encerrando o script.")
else:
  print("Inicializando componentes Haystack para indexação...")
  
  print(f"DocumentEmbedder inicializado com modelo: {document_embedder.model} e URL: {document_embedder.url}")
  document_embedder.run(haystack_docs)
  print("Documentos preparados para indexação.")
  
  # Opcional: Limpeza e Divisão de Documentos
  # cleaner = DocumentCleaner() # Remove cabeçalhos/rodapés, linhas em branco etc. (pode não ser necessário)
  # splitter = DocumentSplitter(split_by="word", split_length=200, split_overlap=20) # Divide documentos longos

  writer: DocumentWriter = DocumentWriter(
    document_store=document_store_for_raw_boardgames,
    policy=DuplicatePolicy.OVERWRITE,
  )
  
  # Pipeline de indexação
  indexing_pipeline: Pipeline = Pipeline()
  # Adicione cleaner e splitter se decidir usá-los:
  # indexing_pipeline.add_component("cleaner", cleaner)
  # indexing_pipeline.add_component("splitter", splitter)
  indexing_pipeline.add_component("embedder", document_embedder)
  indexing_pipeline.add_component("writer", writer)
  
  indexing_pipeline.connect("embedder.documents", "writer.documents")
  
  print("Executando pipeline de indexação...")
  
  batch_size: int = 256
  
  print(f"Processando {len(haystack_docs)} documentos em lotes de {batch_size}...")
  for i in tqdm(range(0, len(haystack_docs), batch_size), desc="Indexando documentos"):
    batch_docs: List[Document] = haystack_docs[i:i + batch_size]
    indexing_pipeline.run({
        "embedder": {"documents": batch_docs},
    })
    print(f"Processando lote {i // batch_size + 1} de {len(haystack_docs) // batch_size + 1}...") 
    print(f"Lote {i // batch_size + 1} processado. Total de documentos indexados: {document_store_for_raw_boardgames.count_documents()}")
  print("Pipeline de indexação concluída.")
  print(f"Total de documentos no ChromaDB ({document_store_for_raw_boardgames._collection_name}): {document_store_for_raw_boardgames.count_documents()}")

Inicializando o ChromaDocumentStore...
ChromaDocumentStore inicializado com persist_path: ./chroma_db_raw_boardgames e collection_name: raw_boardgames
ChromaDocumentStore usando path: ./chroma_db_jogos. Documentos existentes: 1000
Conectando ao MySQL: host=localhost, port=3306, user=root, db=board_games_db
Buscando jogos do MySQL...
Total de 1000 jogos buscados do MySQL.
Preparando Documentos Haystack...


Convertendo dados para Documentos: 100%|██████████| 1000/1000 [00:00<00:00, 51284.51it/s]


Total de 1000 Documentos Haystack preparados.

Amostra do primeiro Documento Haystack:
Content: Nome do jogo: Old Dragon 2ª Edição: Livro I - Regras Básicas. Tipo: b. Adequado para maiores de 12 anos. Pode ser jogado por 1 a 10 jogadores. Tempo médio de jogo: 120 minutos. Temas: Aventura, Fantasia, Literatura, Medieval. Mecânicas: Narração de Histórias, Papel e Caneta, Rolagem de Dados, RPG....
Meta: {'mysql_id': 1, 'title': 'Old Dragon 2ª Edição: Livro I - Regras Básicas', 'min_age': 12, 'max_players': 10, 'min_players': 1, 'play_time_minutes': 120, 'thumbnail': 'https://storage.googleapis.com/ludopedia-capas/43761_t.jpg', 'game_type': 'b', 'categories_list': [], 'themes_list': ['Aventura', 'Fantasia', 'Literatura', 'Medieval'], 'mechanics_list': ['Narração de Histórias', 'Papel e Caneta', 'Rolagem de Dados', 'RPG'], 'artists_list': [], 'favorite_count': 69, 'played_count': 74, 'want_count': 57, 'have_count': 204, 'had_count': 1}
Conexão MySQL fechada.
Inicializando componentes Haysta

Calculating embeddings: 100%|██████████| 32/32 [07:01<00:00, 13.18s/it]


Documentos preparados para indexação.
Executando pipeline de indexação...
Processando 1000 documentos em lotes de 256...


...

Processando lote 2 de 4...
Lote 2 processado. Total de documentos indexados: 1000


Calculating embeddings: 100%|██████████| 8/8 [02:00<00:00, 15.03s/it]
Document c38d09186248e9df3b64a4290fc7c46c754a6dfc2a037b1ee9af2f1b002ad515 contains `meta` values of unsupported types for the keys: categories_list, themes_list, mechanics_list, artists_list. These items will be discarded. Supported types are: str, int, float, bool.
Document 29d5924f3d115b531fa3d213bd521e37b788a9e6a6dd0215c12dcd20cfaa95d3 contains `meta` values of unsupported types for the keys: categories_list, themes_list, mechanics_list, artists_list. These items will be discarded. Supported types are: str, int, float, bool.
Document 983e354e80e5510132e2381ff009af135da3d95a3bc67fb095d558a106e9a585 contains `meta` values of unsupported types for the keys: categories_list, themes_list, mechanics_list, artists_list. These items will be discarded. Supported types are: str, int, float, bool.
Document f3e66b463d45a1946bd47c951d7348fb004ac295cb6f1a9ef8ebe4d1342f5b2b contains `meta` values of unsupported types for the key

Processando lote 3 de 4...
Lote 3 processado. Total de documentos indexados: 1000


Calculating embeddings: 100%|██████████| 8/8 [01:14<00:00,  9.28s/it]
Document 2b727da3995d61a0fe2f7bf606458dad6733eeba8ad538b3a2c841901788f7fe contains `meta` values of unsupported types for the keys: categories_list, themes_list, mechanics_list, artists_list. These items will be discarded. Supported types are: str, int, float, bool.
Document 782f8737a375fcaa475cf5d598082f0ce8059105f4a110405e668858ca176c48 contains `meta` values of unsupported types for the keys: categories_list, themes_list, mechanics_list, artists_list. These items will be discarded. Supported types are: str, int, float, bool.
Document 0955375f57eca9978ad8e8df2a74dbaf644a46761a62c9de83fcf8b790528276 contains `meta` values of unsupported types for the keys: categories_list, themes_list, mechanics_list, artists_list. These items will be discarded. Supported types are: str, int, float, bool.
Document 3adaa273e4f2efe59195ba316f65c42e953754400d07c1208fb8f1555c8ad07a contains `meta` values of unsupported types for the key

Processando lote 4 de 4...
Lote 4 processado. Total de documentos indexados: 1000
Pipeline de indexação concluída.
Total de documentos no ChromaDB (raw_boardgames): 1000


In [ ]:
print(f"ChromaDocumentStore para teste de query inicializado com persist_path: {document_store_for_raw_boardgames._persist_path} e collection_name: {document_store_for_raw_boardgames._collection_name}")

query_text_retriever: ChromaQueryTextRetriever = ChromaQueryTextRetriever(
    document_store=document_store_for_raw_boardgames,
)

print("\nTestando Opção 2: OllamaTextEmbedder + ChromaEmbeddingRetriever")
text_embedder = OllamaTextEmbedder(
    model="nomic-embed-text", 
    url="http://localhost:11434"
)

embedding_retriever = ChromaEmbeddingRetriever(document_store=document_store_for_raw_boardgames)

querying_pipeline_option2: Pipeline = Pipeline()
querying_pipeline_option2.add_component("text_embedder", text_embedder)
querying_pipeline_option2.add_component("embedding_retriever", embedding_retriever)
querying_pipeline_option2.connect("text_embedder.embedding", "embedding_retriever.query_embedding")

results_option2 = querying_pipeline_option2.run({
    "text_embedder": {"text": "jogo de estratégia para dois jogadores com tema medieval"},
    "embedding_retriever": {"top_k": 3}
})

if results_option2["embedding_retriever"]["documents"]:
    print("Resultados da Opção 2:")
    for doc in results_option2["embedding_retriever"]["documents"]:
        print(f"  Title: {doc.meta.get('title', 'N/A')}, Score: {doc.score:.4f}")
        print(f"  Content snippet: {doc.content[:200]}...")
        print(f"  Meta: {doc.meta}")
        print("\n")
else:
    print("Nenhum resultado encontrado pela Opção 2.")

ChromaDocumentStore para teste de query inicializado com persist_path: ./chroma_db_raw_boardgames e collection_name: raw_boardgames

Testando Opção 2: OllamaTextEmbedder + ChromaEmbeddingRetriever
Resultados da Opção 2:
  Title: Torres, Score: 399.5052
  Content snippet: Nome do jogo: Torres. Tipo: b. Adequado para maiores de 12 anos. Pode ser jogado por 2 a 4 jogadores. Tempo médio de jogo: 60 minutos. Categorias: Estratégia Abstrata. Temas: Medieval. Mecânicas: Cerc...
  Meta: {'max_players': 4, 'thumbnail': 'https://storage.googleapis.com/ludopedia-capas/939_t.jpg', 'title': 'Torres', 'favorite_count': 24, 'min_age': 12, 'have_count': 329, 'had_count': 121, 'mysql_id': 854, 'want_count': 331, 'game_type': 'b', 'min_players': 2, 'played_count': 514, 'play_time_minutes': 60}


  Title: The Castles of Burgundy – Edição Especial, Score: 399.5975
  Content snippet: Nome do jogo: The Castles of Burgundy – Edição Especial. Tipo: b. Adequado para maiores de 12 anos. Pode ser jogado por 1 a 